## Uniform Crime Reporting Program - FBI

Scrape data from https://ucr.fbi.gov/ucr-publications corresponding to the number of murders in certain metropolitan regions from 2006 to 2016. For each year, you will programmatically navigate to violent crimes, and then to murders, and scrape the table corresponding to Metropolitan Statistical Areas (MSAs). Along with the numerical data, you will also want to scrape and save the text descriptions on violent crimes and murders.

In [1]:
import pandas as pd
import urllib
import numpy as np
import warnings
from fuzzywuzzy import process, fuzz
warnings.filterwarnings('ignore')

In [2]:
"""
function
-----------
fbi_url_generator

This function pulls violent crime spreadsheets from FBI UCR website
for a given year

It takes in the year of interest and outputs a url string
"""
def fbi_url_generator(year):
    if 2006 <= year <= 2009:
        return('https://www2.fbi.gov/ucr/cius%i/data/documents/'%year +str(year)[2:]+'tbl06.xls')
    else:
        if 2010 <= year <= 2011:
            end = '/tables/table-6/output.xls'
        elif 2012 <= year <= 2013:
            end = '/tables/6tabledatadecpdf/table-6/output.xls'
        elif 2014 <= year <= 2015:
            if year == 2014:
                mid = 'Table_6_Crime_in_the_United_States_by_Metropolitan_Statistical_Area_2014/output.xls'
            else:
                mid = 'table_6_crime_in_the_united_states_by_metropolitan_statistical_area_%i.xls/output.xls' %year
            end = '/tables/table-6/%s' %mid
        elif year == 2016:
            end ='/tables/table-4/table-4/output.xls' 
        hostname = 'https://ucr.fbi.gov/crime-in-the-u.s/%i/crime-in-the-u.s.-%i' %(year, year)
        return(hostname + end)
    

In [3]:
"""
Function
--------
append_df

This function appends two dataframes

Parameters:
    input - dataframe to be appended
    output - dataframe to be appended onto
    
Returns a single dataframe 
"""
def append_df(input,output):
    if output.empty:
        output=input.copy()
    else:
        output=pd.concat([output,input])
        output.reset_index(drop='Index',inplace=True)
    return(output)

In [4]:
"""
# THIS CODE ONLY NEEDS TO BE RUN ONCE TO BRING IN ALL OF THE EXCEL FILES
version='Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36'
test=urllib.request.URLopener()
test.addheader('User-Agent',version)
for year in range(2006, 2017):
    print("Pulling: %i" %year)
    test.retrieve(url=fbi_url_generator(year),filename='data/crime_%i.xls' %year)
"""

'\n# THIS CODE ONLY NEEDS TO BE RUN ONCE TO BRING IN ALL OF THE EXCEL FILES\nversion=\'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36\'\ntest=urllib.request.URLopener()\ntest.addheader(\'User-Agent\',version)\nfor year in range(2006, 2017):\n    print("Pulling: %i" %year)\n    test.retrieve(url=fbi_url_generator(year),filename=\'data/crime_%i.xls\' %year)\n'

In [5]:
#Define empty dataframes
population=pd.DataFrame()
msa_description=pd.DataFrame()
total_crime_by_MSA=pd.DataFrame()
crime_rate_by_MSA=pd.DataFrame()
crime_by_city_in_MSA=pd.DataFrame()

In [6]:
df_allyears = pd.DataFrame()
for year in range(2006, 2017):
    df = pd.read_excel("data/crime_%i.xls" %year,skiprows=[0,1],header=1)

    #######
    # NOTE - misc column has msa population, city population and estimate percentage
    ######
    df=df.iloc[:,0:12] 
    df.columns=['MSA', 'counties','misc', 'violent_crime','mur_mans', 'rape', 'robbery',
                'assault', 'property', 'burglary', 'larceny','mv_theft']

    # Drop footnotes
    footnotes = df['MSA'].str[0].str.isdigit().fillna(False)
    df = df.loc[~footnotes, :]
    
    #Drop blank rows
    df = df.dropna(how='all')

    # Get rid of numbers in MSA
    df['MSA'] = df['MSA'].str.replace('\d+', '')
    # Set empty columns to NaN for MSA
    df['MSA'] = df['MSA'].replace(' ', np.nan, regex=False)
    
    # Sometimes city  names get put in MSA column
    # Messes up carry forward
    df.loc[df['MSA'].str.contains("City of").fillna(False), "MSA"] = np.nan

    # Carry MSA name forward to fill in for all cells
    df.loc[:,'MSA'] = df.loc[:, 'MSA'].fillna(method='ffill')

    ##############
    # POPULATION - grab population and fill in for all MSA
    ##############
    pop_row = df.counties.isnull()
    pop = df.loc[pop_row, ["MSA", 'misc']]
    pop = pop.rename(index=str, columns={'misc': 'msa_pop'})
    
    # Merge population back in
    df = df.loc[~pop_row, :]
    df = df.merge(pop, how='outer', on='MSA')

    ################
    # Descriptions - don't need county descriptions 
    ################
    df = df.loc[df.counties.str.contains("Includes") == False, :]


    ###########################################
    # GOING LONG TO WIDE FOR CRIME VARIABLES
    ###########################################
    crime_vars = ['violent_crime','mur_mans', 'rape', 'robbery',
                  'assault', 'property', 'burglary', 'larceny','mv_theft']

    #########
    # CITIES
    #########
    city_vars = ['MSA', 'counties', 'misc'] + crime_vars
    # Split data Frame
    cities = df.counties.str.contains("City")
    city_df = df.loc[cities, city_vars]
    city_df = city_df.rename(index=str, columns={'misc': 'city_pop'})
    
    # Grab largest city for each MSA and merge back on
    city_df = city_df.sort_values(['MSA','city_pop'], ascending=False)
    large_city = city_df.groupby('MSA').first().reset_index()

    # Rename crime variables to denote city only crime 
    large_city.columns = ['MSA', 'counties', 'city_pop'] + ['city_' + i for i in crime_vars]
    large_city = large_city.rename(index=str, columns={'counties':'largest_city'})
    # Get rid of "City of"
    large_city.loc[:,'largest_city'] = large_city.loc[:, 'largest_city'].str.replace('City of','')
    
    # Merge back to main dataframe
    df = df.loc[~cities, ]
    df = df.merge(large_city, how='outer', on='MSA')

    ###############
    # CRIME RATE
    ###############
    rates = df.counties.str.contains("Rate per")
    rate_vars = ['MSA'] + crime_vars
    rates_df = df.loc[rates, rate_vars]
    rates_df.columns = ['MSA'] + ['rate_' + i for i in crime_vars]

    df = df.loc[~rates, :]
    df = df.merge(rates_df, how='outer', on='MSA')

    ########################
    # MSA-WIDE CRIME STATS
    ########################

    # If the entire MSA reported then there is just one row of numbers
    # If the entire MSA did not report, then there are two rows
            # first row is areas that reported
            # second report is an estimated total
    # We are going to grab the estimates total so our data
    # reflects all areas for all MSA

    # Create Flag for those that do not have complete coverage
    # and are thus estimates
    mins = df.groupby('MSA').misc.min().reset_index()
    mins.columns = ['MSA', 'min_coverage']
    df = df.merge(mins, how='outer', on='MSA')
    df['estimate'] = 0
    df.loc[df.min_coverage < 1, 'estimate'] = 1
    del df['min_coverage']

    # Now only keeping rows with coverage = 1
    # will either be all area or the estimate for all area
    df = df.loc[df.misc == 1, :]

    # Now no longer need coverage or whether its estimate or not
    del df['misc']
    del df['counties']
    
    df['year'] = year
    
    # Append to existing Frame
    df_allyears = append_df(df, df_allyears)

In [7]:
df_allyears = df_allyears.sort_values(["MSA", 'year'])
print(df_allyears.columns)
df_allyears.head()

Index(['MSA', 'violent_crime', 'mur_mans', 'rape', 'robbery', 'assault',
       'property', 'burglary', 'larceny', 'mv_theft', 'msa_pop',
       'largest_city', 'city_pop', 'city_violent_crime', 'city_mur_mans',
       'city_rape', 'city_robbery', 'city_assault', 'city_property',
       'city_burglary', 'city_larceny', 'city_mv_theft', 'rate_violent_crime',
       'rate_mur_mans', 'rate_rape', 'rate_robbery', 'rate_assault',
       'rate_property', 'rate_burglary', 'rate_larceny', 'rate_mv_theft',
       'estimate', 'year'],
      dtype='object')


,MSA,violent_crime,mur_mans,rape,robbery,assault,property,burglary,larceny,mv_theft,...,rate_mur_mans,rate_rape,rate_robbery,rate_assault,rate_property,rate_burglary,rate_larceny,rate_mv_theft,estimate,year
0,"Abilene, TX M.S.A.",638,6,75,109,448,5741,1531,3852,358,...,3.7,46.1,67,275.2,3526.9,940.6,2366.4,219.9,0,2006
357,"Abilene, TX M.S.A.",748,10,103,173,462,5657,1606,3708,343,...,6.3,65.3,109.7,292.9,3586.9,1018.3,2351.1,217.5,0,2007
721,"Abilene, TX M.S.A.",716,7,95,188,426,5124,1417,3443,264,...,4.4,59.7,118,267.5,3217.4,889.8,2161.9,165.8,0,2008
1078,"Abilene, TX M.S.A.",741,7,134,146,454,5607,1488,3816,303,...,4.4,83.9,91.5,284.4,3512.5,932.1,2390.5,189.8,0,2009
1456,"Abilene, TX M.S.A.",675,5,78,116,476,5772,1610,3925,237,...,3.1,48.9,72.7,298.3,3617.3,1009,2459.8,148.5,0,2010


## Fuzzy Text Matching

MSA names changed in 2011

In [8]:
#Perform validations for 2014
    #Scenario 1 - Estimates vs Totals
    #Scenario 2 - Largest City Identification
    #Scenario 3 - Rate Validations
pd.set_option("display.max_columns",41)
year_filter=(df_allyears['MSA'].str.contains('Augusta-Richmond County'))&(df_allyears['year']==2014)
df_allyears[year_filter]

,MSA,violent_crime,mur_mans,rape,robbery,assault,property,burglary,larceny,mv_theft,msa_pop,largest_city,city_pop,city_violent_crime,city_mur_mans,city_rape,city_robbery,city_assault,city_property,city_burglary,city_larceny,city_mv_theft,rate_violent_crime,rate_mur_mans,rate_rape,rate_robbery,rate_assault,rate_property,rate_burglary,rate_larceny,rate_mv_theft,estimate,year
2993,"Augusta-Richmond County, GA-SC M.S.A.",1681,39,199,531,912,19363,4679,13271,1413,585946,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,286.9,6.7,34,90.6,155.6,3304.6,798.5,2264.9,241.1,1,2014


In [9]:
#How many unique MSAs are there in total?
print(df_allyears.shape)

(4114, 33)


In [212]:
pd.set_option("display.max_rows",1000)
df_allyears[['MSA','year']].groupby('MSA').count().sort_values('year')

,year
MSA,
"Homosassa Spring, FL M.S.A.",1
"Nashville-Davidson─\nMurfreesboro─Franklin, TN M.S.A.",1
"Nashville-Davidson─Murfreesboro─ Franklin, TN M.S.A.",1
"Nashville-DavidsonMurfreesboro-Franklin, TN M.S.A.",1
"Detroit-Warren-Livonia, MI M.S.A.,",1
"Dothan, AL M.S.A.,",1
"Kankakee-Bradley, IL M.S.A.",1
"New Orleans-Metairie, LA M.S.A.,",1
"Duluth, MN-WI M.S.A.,",1


In [26]:
df_allyears['MSA'].str.replace(r" M.S.A.",'')

0                                             Abilene, TX
357                                           Abilene, TX
721                                           Abilene, TX
1078                                          Abilene, TX
1456                                          Abilene, TX
1824                                          Abilene, TX
2200                                          Abilene, TX
2589                                          Abilene, TX
2973                                          Abilene, TX
3350                                          Abilene, TX
3728                                          Abilene, TX
2582                       Aguadilla-Isabela, Puerto Rico
2966                       Aguadilla-Isabela, Puerto Rico
3343                       Aguadilla-Isabela, Puerto Rico
3721                       Aguadilla-Isabela, Puerto Rico
4107                       Aguadilla-Isabela, Puerto Rico
349          Aguadilla-Isabela-San Sebastian, Puerto Rico
713          A

In [183]:
filter=df_allyears['MSA'].str.contains("M.D.")
print(df_allyears.shape)
a=df_allyears[~filter]
print(a.shape)

(4114, 33)
(3820, 33)


In [184]:
b=a[['MSA','year']]

In [185]:
b.drop_duplicates(inplace=True)

In [186]:
b.shape

(3820, 2)

In [375]:
state_MSA_lkp=pd.read_csv('state_MSA.csv',header=None)
state_MSA_lkp.sort_values(2)
MSA_lkp=state_MSA_lkp.iloc[:,2:4]
MSA_lkp.drop_duplicates(inplace=True)
MSA_lkp.iloc[:,-1]=MSA_lkp.iloc[:,-1].str.replace(" \(Metropolitan Statistical Area\)",'')

In [376]:
b['MSA']=b['MSA'].str.replace(' M.S.A.','')

In [377]:

def fuzzy_match(input):
    choices = list(MSA_lkp.iloc[:,-1])
    result=process.extractOne(input, choices,scorer=fuzz.token_set_ratio)[0]
    return(result)

In [378]:
b['Match_tknset_ratio']=b['MSA'].apply(lambda s:fuzzy_match(s))

In [199]:
pr=b['MSA'].str.contains('Puerto Rico')
b=b[~pr]
b.shape

(3745, 3)

In [213]:
b.groupby('Match_tknset_ratio').count().sort_values('year')

,MSA,year
Match_tknset_ratio,,
"Carbondale-Marion, IL",1,1
"Sierra Vista-Douglas, AZ",2,2
"Enid, OK",2,2
"Bloomsburg-Berwick, PA",3,3
"Grants Pass, OR",3,3
"Terre Haute, IN",3,3
"Walla Walla, WA",4,4
"Weirton-Steubenville, WV-OH",4,4
"Midland, MI",4,4


In [374]:
c=b.groupby('Match_tknset_ratio').count().sort_values('year')
d=list(c[c['MSA']==11].index)
e=b[b['Match_tknset_ratio'].isin(d)]
del e['year']
e.drop_duplicates(inplace=True)
e.groupby(['Match_tknset_ratio','MSA']).count()#.sort_values('MSA', ascending=False)

,
Match_tknset_ratio,MSA
"Abilene, TX","Abilene, TX"
"Albuquerque, NM","Albuquerque, NM"
"Altoona, PA","Altoona, PA"
"Amarillo, TX","Amarillo, TX"
"Ames, IA","Ames, IA"
"Anchorage, AK","Anchorage, AK"
"Ann Arbor, MI","Ann Arbor, MI"
"Appleton, WI","Appleton, WI"
"Athens-Clarke County, GA","Athens-Clarke County, GA"


In [373]:
b[b['Match_tknset_ratio']=='Carbondale-Marion, IL']

,MSA,year,Match_tknset_ratio
2642,"Carbondale-Marion, IL",2013,"Carbondale-Marion, IL"


In [140]:
b['MSA']=b['MSA'].str.replace(' M.S.A.','')